# OECD HAN Database

In [30]:
# load packages 
library(pacman)
p_load(
    tidyverse, data.table, dtplyr, reshape2, 
    archive, kableExtra, SPARQL, janitor)
# set option
options(dplyr.summarise.inform = FALSE)
gray_scale <- c('#F3F4F8','#D2D4DA', '#B3B5BD', 
                '#9496A1', '#7d7f89', '#777986', 
                '#656673', '#5B5D6B', '#4d505e',
                '#404352', '#2b2d3b', '#282A3A',
                '#1b1c2a', '#191a2b',
                '#141626', '#101223')

In [2]:
han_names <- fread('work/notebooks/patent/data/202208_HAN_NAMES.txt')

In [7]:
dim(han_names)

[1] 4191007       3

In [7]:
head(han_names)

HAN_ID,Clean_name,Person_ctry_code
<int>,<chr>,<chr>
1,& HAMBOURG NIENDORF,DE
2,& KK,JP
3,“ASTRONIT” CLOSE CORP,RU
4,“DEUTSCHE SEE” GMBH,DE
5,“EFIRNOIE” OPEN JOINT STOCK CO,RU
6,“EUROSTANDART” LTD LIABILITY CO,RU


In [3]:
de_firms <- fread('work/notebooks/patent/data/orbis_de_matched_l.csv')

In [55]:
han_names %>%
    .[1:5,] %>%
    kable('pipe', align='ccc')



| HAN_ID |           Clean_name           | Person_ctry_code |
|:------:|:------------------------------:|:----------------:|
|   1    |      & HAMBOURG NIENDORF       |        DE        |
|   2    |              & KK              |        JP        |
|   3    |     “ASTRONIT” CLOSE CORP      |        RU        |
|   4    |      “DEUTSCHE SEE” GMBH       |        DE        |
|   5    | “EFIRNOIE” OPEN JOINT STOCK CO |        RU        |

In [53]:
de_firms %>%
    .[1:5, 2:3]

name_native,name_internat
<chr>,<chr>
Airbus Defence and Space GmbH,Airbus Defence and Space GmbH
EurA AG,EurA AG
TuTech Innovation GmbH,TuTech Innovation GmbH
FFT Produktionssysteme GmbH & Co. KG.,FFT Produktionssysteme GmbH & Co. KG.
Diehl Aviation Laupheim GmbH,Diehl Aviation Laupheim GmbH


In [83]:
# query
airbus <- toupper('Airbus Defence')
han_names %>%
    .[Person_ctry_code == 'DE'] %>%
    .[Clean_name %like% airbus] %>%
    .[,HAN_ID] -> airbus_han_ids
airbus_han_ids

[1]   60513   62422 3637004 4401227 4527012

In [61]:
han_patents <- fread('work/notebooks/patent/data/202208_HAN_PATENTS.txt')

In [86]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[, .N, by=Publn_auth] -> foo
    transform(adorn_totals(foo)) %>%
    transpose() %>%
    row_to_names(row_number=1)

,EP,US,WO,Total
,<chr>,<chr>,<chr>,<chr>
2,716,415,88,1219


In [84]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[sample(.N, 5)]

HAN_ID,HARM_ID,Appln_id,Publn_auth,Patent_number
<int>,<int>,<int>,<chr>,<chr>
60513,60513,442096647,EP,EP3112597
60513,60513,417401751,EP,EP2913271
60513,60513,404985890,EP,EP2825450
60513,60513,527283926,US,US2020070202
60513,60513,544408405,WO,WO2021123759


In [26]:
fooids = c('US2017165795', 'EP3181711', 'US10703486', 'EP3219614', 'US9308691','EP2689872')
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[Patent_number %in% fooids] %>%
    kable('pipe', align='ccccl')



| HAN_ID | HARM_ID | Appln_id  | Publn_auth |Patent_number |
|:------:|:-------:|:---------:|:----------:|:-------------|
| 60513  |  60513  | 408666014 |     EP     |EP2689872     |
| 60513  |  60513  | 415684469 |     US     |US9308691     |
| 60513  |  60513  | 450308171 |     EP     |EP3219614     |
| 60513  |  60513  | 480021212 |     US     |US2017165795  |
| 60513  |  60513  | 483683605 |     US     |US10703486    |

In [34]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[Publn_auth == 'EP'] %>%
    head %>%
    kable('pipe', align='ccccc')



| HAN_ID | HARM_ID | Appln_id | Publn_auth | Patent_number |
|:------:|:-------:|:--------:|:----------:|:-------------:|
| 60513  |  60513  |   213    |     EP     |   EP2030891   |
| 60513  |  60513  |  65448   |     EP     |   EP2025928   |
| 60513  |  60513  |  156990  |     EP     |   EP1920908   |
| 60513  |  60513  |  161551  |     EP     |   EP1972896   |
| 60513  |  60513  |  173385  |     EP     |   EP2134522   |
| 60513  |  60513  |  173386  |     EP     |   EP2136979   |

In [5]:
foo <- fread('work/notebooks/patent/data/resulttable 2.csv')

In [10]:
foo %>%
    .[, c(1, 5, 6, 7, 9)] %>%
    kable('pipe', align='ccccc')



| pat_publn_id | publn_kind | appln_id | publn_date | publn_first_grant |
|:------------:|:----------:|:--------:|:----------:|:-----------------:|
|  277148936   |     A1     |  156990  | 2008-05-14 |         N         |
|  437725522   |     B1     |  156990  | 2015-04-08 |         Y         |

In [19]:
endpoint <- "https://data.epo.org/linked-data/query"
# create query statement
query <- "PREFIX skos: <http://www.w3.org/2004/02/skos/core#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#> 
prefix patent: <http://data.epo.org/linked-data/def/patent/>

SELECT * 
WHERE {
   ?publn rdf:type patent:Publication;
          patent:applicantVC ?applicant;
          patent:publicationAuthority/skos:notation ?auth;
          patent:publicationNumber ?nr;
          patent:publicationKind/skos:notation ?kind.
  ?applicant vcard:fn ?name.
  
   FILTER(STRSTARTS(UCASE(?name), 'HUAWEI'))
} LIMIT 10"
# Step 2 - Use SPARQL package to submit query and save results to a data frame
qd <- SPARQL(endpoint,query,curl_args=list(useragent=R.version.string))
df <- qd$results

In [20]:
head(df)

,publn,applicant,auth,nr,kind,name
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,<http://data.epo.org/linked-data/data/publication/EP/1536591/A1/->,<http://data.epo.org/linked-data/data/vc/AF6A00EBED9A0A944405571B3CD93799>,EP,1536591,A1,"Huawei Technologies Co., Ltd."
2,<http://data.epo.org/linked-data/data/publication/EP/1566921/A1/->,<http://data.epo.org/linked-data/data/vc/CDC89188C00CD3068260AFCD99E44CD3>,EP,1566921,A1,"Huawei Technologies Co., Ltd."
3,<http://data.epo.org/linked-data/data/publication/EP/2114028/B1/->,<http://data.epo.org/linked-data/data/vc/30A7AAC966CA77F2EA95DBAB78798573>,EP,2114028,B1,"Huawei Technologies Co., Ltd."
4,<http://data.epo.org/linked-data/data/publication/EP/2114028/A1/->,<http://data.epo.org/linked-data/data/vc/F2806D1A93692FF97F69FEA3009D96DC>,EP,2114028,A1,"Huawei Technologies Co., Ltd."
5,<http://data.epo.org/linked-data/data/publication/EP/2112780/A1/->,<http://data.epo.org/linked-data/data/vc/E981DB78F395D6429045B02441FC54EE>,EP,2112780,A1,"Huawei Technologies Co., Ltd."
6,<http://data.epo.org/linked-data/data/publication/EP/2698967/A1/->,<http://data.epo.org/linked-data/data/vc/2AAA05ADF2FBC02CA97DC72726E181A4>,EP,2698967,A1,"Huawei Technologies Co., Ltd"
